In [ ]:
!pip install scikit-learn tensorflow

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, concatenate, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

# Misalkan Anda memiliki dataset dalam bentuk DataFrame pandas dengan kolom 'user', 'item', dan 'rating'
# Contoh dataset dummy:
data = {'user': np.random.randint(1, 100, 1000),
        'item': np.random.randint(1, 50, 1000),
        'rating': np.random.randint(1, 6, 1000)}
df = pd.DataFrame(data)

# Label encoding untuk kolom 'user' dan 'item'
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['user'] = user_encoder.fit_transform(df['user'])
df['item'] = item_encoder.fit_transform(df['item'])

# Membagi dataset menjadi set pelatihan dan validasi
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Menentukan jumlah pengguna dan item serta ukuran embedding
n_users = df['user'].nunique()
n_items = df['item'].nunique()
embedding_size = 50

# Model Neural Collaborative Filtering (NCF)
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

user_embedding = Embedding(n_users + 1, embedding_size, input_length=1)(user_input)
item_embedding = Embedding(n_items + 1, embedding_size, input_length=1)(item_input)

merged_vector = concatenate([user_embedding, item_embedding])
flatten = Flatten()(merged_vector)

dense_layer = Dense(64, activation='relu')(flatten)
output_layer_ncf = Dense(1, activation='linear', name='output_ncf')(dense_layer)

ncf_model = Model(inputs=[user_input, item_input], outputs=output_layer_ncf)
ncf_model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

# Melatih model
ncf_model.fit([train['user'], train['item']], train['rating'], batch_size=32, epochs=10, validation_split=0.2)

# Memprediksi rating pada set validasi
predictions = ncf_model.predict([test['user'], test['item']])

# Mengukur kinerja model menggunakan MAE dan RMSE
mae = mean_absolute_error(test['rating'], predictions)
rmse = np.sqrt(mean_squared_error(test['rating'], predictions))

print(f'MAE on test set: {mae}')
print(f'RMSE on test set: {rmse}')


Epoch 1/10
20/20 [==============================] - 1s 10ms/step - loss: 10.2403 - val_loss: 9.8463
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 8.3850 - val_loss: 7.2456
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 5.1107 - val_loss: 3.5561
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 2.3893 - val_loss: 2.3108
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 1.9526 - val_loss: 2.3628
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 1.8057 - val_loss: 2.4675
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 1.7595 - val_loss: 2.5888
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 1.7344 - val_loss: 2.6844
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 1.7192 - val_loss: 2.7555
Epoch 10/10
7/7 [==============================] - 0s 2ms/step
MAE on test set: 1.325759310722351
RMSE on test set: 